In [24]:
import numpy as np
import pandas as pd
import geonamescache 
import unicodedata
import unidecode
import re
import multiprocessing as mp
import os
from multiprocessing import Pool, TimeoutError
from pandas.io.json import json_normalize

gc = geonamescache.GeonamesCache()

# read data from file
df = pd.read_csv('./data/headlines.txt', sep="\n", header=None, names=['headline'])

# get cities from geonamescache and sort them by string-length descending
cities = gc.get_cities()
known_cities=list(map(lambda k: cities[k]['name'], cities)) 
known_cities=list(map(lambda v: unidecode.unidecode(v), known_cities))
known_cities.sort(key=lambda k: len(k),reverse=True)

# get countries in same way
countries = gc.get_countries()
known_countries=list(map(lambda k: countries[k]['name'], countries))
known_countries=list(map(lambda v: unidecode.unidecode(v), known_countries))
known_countries.sort(key=lambda k: len(k),reverse=True)

# df is a working data frame
df = df.assign(cities=None)
df['cities'] =  [[] for _ in range(len(df))]
df['countries'] =  [[] for _ in range(len(df))]

print( 'number of cities = ', len(known_cities))
print( 'number of countries = ', len(known_countries))

list_of_cities=list(map(str, known_cities))
list_of_countries=list(map(str, known_countries))

city_pattern=re.compile('|'.join(map(lambda x: '\\b' + re.escape(x) + '\\b', list_of_cities)),flags=re.IGNORECASE+re.MULTILINE)
country_pattern=re.compile('|'.join(map(lambda x: '\\b' + re.escape(x) + '\\b', list_of_countries)),flags=re.IGNORECASE+re.MULTILINE)

for index, row in df.iterrows(): 
    items = re.findall( city_pattern, row['headline'] )
    for item in items:
        row['cities'].append(item)
    items = re.findall( country_pattern, row['headline'] )
    for item in items:
        row['countries'].append(item)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)

df['city']='-'
df['country']='-'

for index, row in df.iterrows():
    list_of_cities = row['cities']
    if len(list_of_cities) > 0:
        list_of_cities.sort(key=lambda k: len(k), reverse=True)
        row['city'] = list_of_cities[0]
    list_of_countries = row['countries']
    if len(list_of_countries) > 0:
        list_of_countries.sort(key=lambda k: len(k), reverse=True)
        row['country'] = list_of_countries[0]
    current_city = row['city']
    current_country= row['country']
    if current_country != '-':
        tmp = gc.get_countries_by_names()
        print(tmp)
    # todo
    #city_matches = gc.get_cities_by_name( row['city'])
    #if len(city_matches) > 1:
    #    print( current_city, ' ## ', city_matches )
df


number of cities =  24336
number of countries =  252
{'Andorra': {'geonameid': 3041565, 'name': 'Andorra', 'iso': 'AD', 'iso3': 'AND', 'isonumeric': 20, 'fips': 'AN', 'continentcode': 'EU', 'capital': 'Andorra la Vella', 'areakm2': 468, 'population': 84000, 'tld': '.ad', 'currencycode': 'EUR', 'currencyname': 'Euro', 'phone': '376', 'postalcoderegex': '^(?:AD)*(\\d{3})$', 'languages': 'ca', 'neighbours': 'ES,FR'}, 'United Arab Emirates': {'geonameid': 290557, 'name': 'United Arab Emirates', 'iso': 'AE', 'iso3': 'ARE', 'isonumeric': 784, 'fips': 'AE', 'continentcode': 'AS', 'capital': 'Abu Dhabi', 'areakm2': 82880, 'population': 4975593, 'tld': '.ae', 'currencycode': 'AED', 'currencyname': 'Dirham', 'phone': '971', 'postalcoderegex': '', 'languages': 'ar-AE,fa,en,hi,ur', 'neighbours': 'SA,OM'}, 'Afghanistan': {'geonameid': 1149361, 'name': 'Afghanistan', 'iso': 'AF', 'iso3': 'AFG', 'isonumeric': 4, 'fips': 'AF', 'continentcode': 'AS', 'capital': 'Kabul', 'areakm2': 647500, 'population':

,headline,cities,countries,city,country
0,Zika Outbreak Hits Miami,[Miami],[],Miami,-
1,Could Zika Reach New York City?,[New York City],[],New York City,-
2,First Case of Zika in Miami Beach,"[Miami Beach, of]",[],Miami Beach,-
3,"Mystery Virus Spreads in Recife, Brazil",[Recife],[Brazil],Recife,Brazil
4,Dallas man comes down with case of Zika,"[Dallas, man, of]",[],Dallas,-
5,Trinidad confirms first Zika case,[Trinidad],[],Trinidad,-
6,Zika Concerns are Spreading in Houston,[Houston],[],Houston,-
7,Geneve Scientists Battle to Find Cure,[Geneve],[],Geneve,-
8,The CDC in Atlanta is Growing Worried,[Atlanta],[],Atlanta,-
9,Zika Infested Monkeys in Sao Paulo,[Sao Paulo],[],Sao Paulo,-
